In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader,Dataset
import os
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import copy
from PIL import Image

In [2]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
test_dir=r'E:\Dance-hackerearth\DanceForms\test'
train_dir=r'E:\Dance-hackerearth\Forms\train'
val_dir=r'E:\Dance-hackerearth\Forms\val'

In [4]:
test=pd.read_csv(r"E:\Dance-hackerearth\DanceForms\test.csv")
train=pd.read_csv(r"E:\Dance-hackerearth\DanceForms\train.csv")
print(len(train),"|",len(test))

364 | 156


In [5]:
lb=LabelEncoder()
lb.fit(train.Target.unique())
categories=lb.classes_
categories

array(['bharatanatyam', 'kathak', 'kathakali', 'kuchipudi', 'manipuri',
       'mohiniyattam', 'odissi', 'sattriya'], dtype=object)

In [6]:
values=lb.fit_transform(train.Target)
values

array([4, 0, 6, 2, 6, 1, 7, 3, 7, 3, 3, 3, 1, 7, 2, 6, 7, 3, 4, 1, 4, 0,
       3, 3, 5, 6, 1, 6, 6, 7, 2, 1, 2, 7, 0, 1, 0, 2, 6, 6, 6, 3, 4, 5,
       5, 6, 1, 4, 1, 5, 6, 7, 7, 6, 3, 7, 5, 4, 7, 1, 6, 7, 4, 2, 7, 2,
       4, 0, 1, 1, 7, 0, 2, 1, 5, 4, 1, 3, 6, 3, 0, 3, 0, 7, 1, 3, 5, 4,
       5, 0, 6, 3, 6, 3, 0, 5, 0, 0, 7, 1, 2, 5, 7, 0, 6, 1, 7, 7, 3, 6,
       1, 4, 2, 0, 5, 2, 3, 4, 5, 0, 5, 1, 2, 3, 0, 5, 0, 5, 2, 0, 5, 0,
       4, 4, 7, 5, 7, 0, 6, 5, 6, 5, 2, 6, 7, 0, 5, 6, 7, 4, 7, 7, 1, 6,
       2, 0, 5, 1, 3, 4, 7, 2, 5, 2, 1, 7, 5, 0, 0, 3, 6, 1, 6, 3, 5, 3,
       6, 7, 0, 0, 2, 6, 5, 4, 6, 2, 5, 5, 3, 1, 5, 4, 6, 6, 3, 2, 0, 2,
       5, 0, 2, 5, 6, 4, 3, 0, 7, 3, 2, 5, 2, 7, 5, 1, 3, 5, 3, 7, 5, 1,
       5, 7, 6, 1, 1, 2, 5, 5, 7, 1, 0, 1, 4, 2, 7, 4, 3, 4, 1, 0, 1, 3,
       2, 2, 6, 3, 0, 2, 3, 2, 6, 0, 7, 7, 1, 0, 6, 7, 0, 6, 7, 5, 2, 2,
       3, 3, 6, 4, 6, 7, 1, 6, 4, 0, 3, 0, 7, 5, 4, 0, 2, 5, 0, 1, 2, 3,
       0, 2, 5, 2, 3, 7, 7, 5, 2, 7, 7, 3, 4, 2, 4,

In [7]:
train_transforms=transforms.Compose([transforms.Resize(256),  
                                     torchvision.transforms.CenterCrop(224),
                                 transforms.RandomHorizontalFlip(),                                 
                                 torchvision.transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0),
                                 transforms.ToTensor(),
                                 torchvision.transforms.Normalize(mean=[0.5, 0.5,0.5],std=[0.25,0.25,0.25])])

transform_val=torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]),
])

In [8]:
train_images=datasets.ImageFolder(train_dir,transform=train_transforms)
train_images=DataLoader(train_images,batch_size=16,shuffle=True)
train_images

In [9]:
val_images=datasets.ImageFolder(val_dir,transform=transform_val)
val_images=DataLoader(val_images,batch_size=16,shuffle=True)
val_images

In [10]:
class CustomDataset(Dataset):
    def __init__(self,main_dir,transform):
        self.main_dir=main_dir
        self.transform=transform
        self.all_imgs=os.listdir(main_dir)
        

    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self,idx):
        img_loc=os.path.join(self.main_dir,self.all_imgs[idx])
        #print(idx)
        image=Image.open(img_loc)
        tensor_image=self.transform(image)
        return tensor_image,self.all_imgs[idx]

In [11]:
test_loader=CustomDataset(test_dir,transform=transform_val)
test_images=DataLoader(test_loader,batch_size=16,shuffle=False)
test_images

In [12]:
i=0
for img,label in test_images:
    print(img[i],label[i])
    i+=1

tensor([[[-1.9686, -1.9686, -1.9686,  ..., -1.9843, -1.9843, -2.0000],
         [-1.9686, -1.9686, -1.9686,  ..., -1.9843, -1.9843, -2.0000],
         [-1.9686, -1.9686, -1.9686,  ..., -1.9843, -1.9843, -1.9843],
         ...,
         [ 0.0549,  0.0392,  0.0078,  ...,  1.4980,  1.4196,  1.4980],
         [ 0.3686,  0.4784,  0.5255,  ...,  1.5137,  1.5294,  1.5451],
         [ 0.3059,  0.4000,  0.4627,  ...,  1.5294,  1.6078,  1.6549]],

        [[-1.9373, -1.9373, -1.9373,  ..., -1.9529, -1.9529, -1.9686],
         [-1.9373, -1.9373, -1.9373,  ..., -1.9529, -1.9529, -1.9686],
         [-1.9373, -1.9373, -1.9373,  ..., -1.9529, -1.9529, -1.9529],
         ...,
         [-0.8863, -0.9176, -0.9804,  ..., -0.6667, -0.6980, -0.7608],
         [-0.9647, -0.9176, -0.9333,  ..., -0.7137, -0.7765, -0.7451],
         [-0.9647, -0.9333, -0.9020,  ..., -0.7765, -0.7765, -0.7137]],

        [[-1.9843, -1.9843, -1.9843,  ..., -1.9843, -1.9843, -1.9843],
         [-1.9843, -1.9843, -1.9843,  ..., -1

         [ 1.9216,  1.9216,  1.9059,  ...,  1.3098,  1.2157,  1.1216]]]) 416.jpg
tensor([[[-1.8745, -1.8745, -1.8745,  ..., -1.8745, -1.8745, -1.8745],
         [-1.8745, -1.8745, -1.8745,  ..., -1.8745, -1.8745, -1.8745],
         [-1.8745, -1.8745, -1.8745,  ..., -1.8745, -1.8745, -1.8745],
         ...,
         [ 1.1216,  1.2471,  1.2000,  ...,  1.5451,  1.8902,  1.9686],
         [ 1.5137,  1.5765,  1.5137,  ...,  1.8275,  1.9529,  1.9843],
         [ 1.6863,  1.7176,  1.6549,  ...,  1.9686,  1.9373,  1.8431]],

        [[-1.9686, -1.9686, -1.9686,  ..., -1.9686, -1.9686, -1.9686],
         [-1.9686, -1.9686, -1.9686,  ..., -1.9686, -1.9686, -1.9686],
         [-1.9686, -1.9686, -1.9686,  ..., -1.9686, -1.9686, -1.9686],
         ...,
         [-0.3686, -0.2431, -0.2902,  ..., -0.0235,  0.4627,  0.8706],
         [-0.0549,  0.0235, -0.0549,  ...,  0.4784,  0.7765,  0.9176],
         [ 0.0549,  0.1176,  0.0549,  ...,  0.7451,  0.8706,  0.8078]],

        [[-1.9373, -1.9373, -1.9373

In [13]:
model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
for p in model.parameters():
    p.requires_grad=False

In [15]:
model.last_linear=nn.Linear(in_features=512,out_features=8)
last_layer_feat=model.fc.in_features
model.fc=nn.Sequential(nn.Linear(last_layer_feat, 256),
                         nn.ReLU(),
                         nn.Dropout(0.5),
                         nn.Linear(256,len(categories)))
                         
model=model.to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
def training_time(model,criterion,optimizer,scheduler,num_epochs=80):
    model.train()
    best_model_stat=copy.deepcopy(model.state_dict())
    best_accuracy=0.0
    for epoch in range(num_epochs):
        print("Epoch ",epoch+1,"/",num_epochs,'\t')
        
        running_loss=0.0
        correct=0
        total=0
        print("Train-Data:")
        for batch_idx,(inputs,labels) in enumerate(train_images):
            inputs=inputs.to(device)
            labels=labels.to(device)
            optimizer.zero_grad()
            outputs=model(inputs)
            _,preds=outputs.max(1)
            loss=criterion(outputs,labels.long())
            loss.backward()
            optimizer.step()
            #scheduler.step()
            total+=labels.size(0)
            running_loss+=loss.item()            
            correct+=preds.eq(labels).sum().item()
            
        final_loss=running_loss/(batch_idx+1)
        acc=(correct/total)*100.00
        
        print('Loss : {:.4f} | Accuracy : {:.4f} | {}/{}'.format(final_loss,acc,correct,total))
        print("Validation-Data")
        model.eval()
        val_loss=0.0
        correct=0
        total=0
        for batch_idx,(inputs,labels) in enumerate(val_images):
            inputs=inputs.to(device)
            labels=labels.to(device)
            outputs=model(inputs)
            loss=criterion(outputs,labels.long())
            val_loss+=loss.item()
            _,preds=torch.max(outputs,1)
            total+=labels.size(0)
            correct+=preds.eq(labels).sum().item()
        final_val_loss=val_loss/(batch_idx+1)
        val_acc=(correct/total)*100.00
        print('Loss : {:.4f} | Accuracy : {:.4f} | {}/{}'.format(final_val_loss,val_acc,correct,total)) 
        if val_acc>best_accuracy:
            best_accuracy=val_acc
            best_model_stat=copy.deepcopy(model.state_dict())
        print("-"*10)
            
    print('Best Accuracy : {:.4f}'.format(best_accuracy))
    model.load_state_dict(best_model_stat)
    return model    

In [17]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.0001,weight_decay=1e-5)
#step_lr=lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)
num_epochs=80

In [18]:
model_trained=training_time(model,criterion,optimizer,num_epochs)

Epoch  1 / 80 	
Train-Data:
Loss : 2.1213 | Accuracy : 15.0134 | 56/373
Validation-Data
Loss : 1.9950 | Accuracy : 22.2222 | 12/54
----------
Epoch  2 / 80 	
Train-Data:
Loss : 1.9605 | Accuracy : 27.8820 | 104/373
Validation-Data
Loss : 1.9494 | Accuracy : 24.0741 | 13/54
----------
Epoch  3 / 80 	
Train-Data:
Loss : 1.8397 | Accuracy : 37.5335 | 140/373
Validation-Data
Loss : 1.8204 | Accuracy : 31.4815 | 17/54
----------
Epoch  4 / 80 	
Train-Data:
Loss : 1.7309 | Accuracy : 42.8954 | 160/373
Validation-Data
Loss : 1.7601 | Accuracy : 38.8889 | 21/54
----------
Epoch  5 / 80 	
Train-Data:
Loss : 1.6179 | Accuracy : 51.7426 | 193/373
Validation-Data
Loss : 1.6861 | Accuracy : 48.1481 | 26/54
----------
Epoch  6 / 80 	
Train-Data:
Loss : 1.5305 | Accuracy : 57.3727 | 214/373
Validation-Data
Loss : 1.5938 | Accuracy : 51.8519 | 28/54
----------
Epoch  7 / 80 	
Train-Data:
Loss : 1.4190 | Accuracy : 63.5389 | 237/373
Validation-Data
Loss : 1.5546 | Accuracy : 51.8519 | 28/54
----------


Loss : 0.2647 | Accuracy : 95.1743 | 355/373
Validation-Data
Loss : 0.6079 | Accuracy : 81.4815 | 44/54
----------
Epoch  59 / 80 	
Train-Data:
Loss : 0.2667 | Accuracy : 93.8338 | 350/373
Validation-Data
Loss : 0.6148 | Accuracy : 87.0370 | 47/54
----------
Epoch  60 / 80 	
Train-Data:
Loss : 0.2429 | Accuracy : 95.4424 | 356/373
Validation-Data
Loss : 0.5727 | Accuracy : 85.1852 | 46/54
----------
Epoch  61 / 80 	
Train-Data:
Loss : 0.2643 | Accuracy : 94.3700 | 352/373
Validation-Data
Loss : 0.5751 | Accuracy : 87.0370 | 47/54
----------
Epoch  62 / 80 	
Train-Data:
Loss : 0.2419 | Accuracy : 95.9786 | 358/373
Validation-Data
Loss : 0.6086 | Accuracy : 81.4815 | 44/54
----------
Epoch  63 / 80 	
Train-Data:
Loss : 0.2481 | Accuracy : 95.7105 | 357/373
Validation-Data
Loss : 0.6809 | Accuracy : 83.3333 | 45/54
----------
Epoch  64 / 80 	
Train-Data:
Loss : 0.2261 | Accuracy : 96.5147 | 360/373
Validation-Data
Loss : 0.5752 | Accuracy : 87.0370 | 47/54
----------
Epoch  65 / 80 	
Trai

In [19]:
torch.save(model_trained.state_dict(), r"E:\Dance-hackerearth\dance_model.pth")

In [20]:
model_trained.eval()
test_predictions=[]
img_l=[]
i=0
for p,label in (test_images):
    p=p.to(device)
    outputs=model_trained.forward(p)
    _, preds=outputs.max(1)    
    test_predictions.append(preds)
    img_l.append(label)
img_l

[('108.jpg',
  '11.jpg',
  '110.jpg',
  '112.jpg',
  '114.jpg',
  '119.jpg',
  '125.jpg',
  '128.jpg',
  '129.jpg',
  '13.jpg',
  '132.jpg',
  '138.jpg',
  '14.jpg',
  '145.jpg',
  '146.jpg',
  '147.jpg'),
 ('150.jpg',
  '152.jpg',
  '157.jpg',
  '162.jpg',
  '17.jpg',
  '170.jpg',
  '171.jpg',
  '173.jpg',
  '175.jpg',
  '183.jpg',
  '189.jpg',
  '193.jpg',
  '194.jpg',
  '196.jpg',
  '198.jpg',
  '20.jpg'),
 ('200.jpg',
  '202.jpg',
  '206.jpg',
  '210.jpg',
  '215.jpg',
  '216.jpg',
  '217.jpg',
  '220.jpg',
  '224.jpg',
  '226.jpg',
  '23.jpg',
  '232.jpg',
  '238.jpg',
  '239.jpg',
  '246.jpg',
  '256.jpg'),
 ('264.jpg',
  '269.jpg',
  '270.jpg',
  '271.jpg',
  '275.jpg',
  '278.jpg',
  '28.jpg',
  '280.jpg',
  '283.jpg',
  '284.jpg',
  '285.jpg',
  '286.jpg',
  '290.jpg',
  '291.jpg',
  '292.jpg',
  '301.jpg'),
 ('302.jpg',
  '306.jpg',
  '316.jpg',
  '317.jpg',
  '318.jpg',
  '323.jpg',
  '324.jpg',
  '33.jpg',
  '330.jpg',
  '332.jpg',
  '34.jpg',
  '340.jpg',
  '346.jpg',
  '3

In [21]:
test_predictions

[tensor([4, 1, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 6, 0, 0, 0]),
 tensor([0, 0, 0, 0, 1, 0, 0, 6, 3, 2, 2, 2, 2, 2, 2, 1]),
 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 5, 5]),
 tensor([5, 5, 5, 5, 7, 5, 1, 5, 5, 5, 5, 7, 5, 5, 5, 5]),
 tensor([5, 5, 7, 6, 7, 7, 0, 1, 1, 7, 1, 7, 7, 7, 6, 7]),
 tensor([7, 1, 4, 7, 3, 3, 6, 0, 1, 3, 6, 6, 3, 6, 3, 7]),
 tensor([1, 0, 6, 3, 3, 6, 0, 0, 7, 3, 4, 1, 3, 3, 0, 3]),
 tensor([6, 6, 1, 0, 5, 6, 6, 6, 6, 0, 6, 6, 6, 3, 5, 6]),
 tensor([6, 3, 6, 6, 0, 6, 3, 0, 0, 6, 6, 6, 6, 3, 1, 1]),
 tensor([7, 1, 7, 1, 1, 4, 4, 4, 1, 4, 4, 4])]

In [22]:
final=[]
for row in test_predictions:
    for element in row:
        final.append(element.item())
final

[4,
 1,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 6,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 7,
 2,
 2,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 7,
 5,
 1,
 5,
 5,
 5,
 5,
 7,
 5,
 5,
 5,
 5,
 5,
 5,
 7,
 6,
 7,
 7,
 0,
 1,
 1,
 7,
 1,
 7,
 7,
 7,
 6,
 7,
 7,
 1,
 4,
 7,
 3,
 3,
 6,
 0,
 1,
 3,
 6,
 6,
 3,
 6,
 3,
 7,
 1,
 0,
 6,
 3,
 3,
 6,
 0,
 0,
 7,
 3,
 4,
 1,
 3,
 3,
 0,
 3,
 6,
 6,
 1,
 0,
 5,
 6,
 6,
 6,
 6,
 0,
 6,
 6,
 6,
 3,
 5,
 6,
 6,
 3,
 6,
 6,
 0,
 6,
 3,
 0,
 0,
 6,
 6,
 6,
 6,
 3,
 1,
 1,
 7,
 1,
 7,
 1,
 1,
 4,
 4,
 4,
 1,
 4,
 4,
 4]

In [23]:
final_labels=[]
for row in img_l:
    for element in row:
        final_labels.append(element)
final_labels

['108.jpg',
 '11.jpg',
 '110.jpg',
 '112.jpg',
 '114.jpg',
 '119.jpg',
 '125.jpg',
 '128.jpg',
 '129.jpg',
 '13.jpg',
 '132.jpg',
 '138.jpg',
 '14.jpg',
 '145.jpg',
 '146.jpg',
 '147.jpg',
 '150.jpg',
 '152.jpg',
 '157.jpg',
 '162.jpg',
 '17.jpg',
 '170.jpg',
 '171.jpg',
 '173.jpg',
 '175.jpg',
 '183.jpg',
 '189.jpg',
 '193.jpg',
 '194.jpg',
 '196.jpg',
 '198.jpg',
 '20.jpg',
 '200.jpg',
 '202.jpg',
 '206.jpg',
 '210.jpg',
 '215.jpg',
 '216.jpg',
 '217.jpg',
 '220.jpg',
 '224.jpg',
 '226.jpg',
 '23.jpg',
 '232.jpg',
 '238.jpg',
 '239.jpg',
 '246.jpg',
 '256.jpg',
 '264.jpg',
 '269.jpg',
 '270.jpg',
 '271.jpg',
 '275.jpg',
 '278.jpg',
 '28.jpg',
 '280.jpg',
 '283.jpg',
 '284.jpg',
 '285.jpg',
 '286.jpg',
 '290.jpg',
 '291.jpg',
 '292.jpg',
 '301.jpg',
 '302.jpg',
 '306.jpg',
 '316.jpg',
 '317.jpg',
 '318.jpg',
 '323.jpg',
 '324.jpg',
 '33.jpg',
 '330.jpg',
 '332.jpg',
 '34.jpg',
 '340.jpg',
 '346.jpg',
 '349.jpg',
 '35.jpg',
 '354.jpg',
 '358.jpg',
 '360.jpg',
 '363.jpg',
 '366.jpg',
 '

In [24]:
dance_names=lb.inverse_transform(final)

In [25]:
dance_names

array(['manipuri', 'kathak', 'manipuri', 'bharatanatyam', 'bharatanatyam',
       'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam',
       'kathak', 'bharatanatyam', 'bharatanatyam', 'odissi',
       'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam',
       'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'kathak',
       'bharatanatyam', 'bharatanatyam', 'odissi', 'kuchipudi',
       'kathakali', 'kathakali', 'kathakali', 'kathakali', 'kathakali',
       'kathakali', 'kathak', 'kathakali', 'kathakali', 'kathakali',
       'kathakali', 'kathakali', 'kathakali', 'kathakali', 'kathakali',
       'kathakali', 'kathakali', 'sattriya', 'kathakali', 'kathakali',
       'kathakali', 'mohiniyattam', 'mohiniyattam', 'mohiniyattam',
       'mohiniyattam', 'mohiniyattam', 'mohiniyattam', 'sattriya',
       'mohiniyattam', 'kathak', 'mohiniyattam', 'mohiniyattam',
       'mohiniyattam', 'mohiniyattam', 'sattriya', 'mohiniyattam',
       'mohiniyattam', 'mohiniy

In [26]:
sub=pd.read_csv(r'E:\Dance-hackerearth\DanceForms\Submissions.csv')

In [27]:
sub.head()

,Image,target
0,108.jpg,manipuri
1,11.jpg,kathak
2,110.jpg,manipuri
3,112.jpg,bharatanatyam
4,114.jpg,bharatanatyam


In [28]:
sub["Image"]=final_labels

In [29]:
sub["target"]=dance_names

In [30]:
sub.head

<bound method NDFrame.head of        Image         target
0    108.jpg       manipuri
1     11.jpg         kathak
2    110.jpg       manipuri
3    112.jpg  bharatanatyam
4    114.jpg  bharatanatyam
..       ...            ...
151   86.jpg       manipuri
152    9.jpg         kathak
153   92.jpg       manipuri
154   95.jpg       manipuri
155   97.jpg       manipuri

[156 rows x 2 columns]>